### MySQL database

Man kan koble opp mot forskjellige databaser i Python:
- GadFly
- mSQL
- **MySQL**
- PostgreSQL
- Microsoft SQL Server 2000
- Informix
- Interbase
- Oracle
- Sybase
- …

Eller bruke SqlLite3 
- Egen database modul for Python



In [2]:
# Viser hvordan vi kobler oss mot MySQL og leser/oppdaterer data.
# Må installere PyMySQL først: pip install pymysql
import pymysql

For å logge oss inn på databasen, må vi ha en konto på database-serveren.

Følgende informasjon må vi ha:

- host: navn på database-server
- user: brukernavn for å koble oss til serveren
- password: passord for å koble oss til serveren
- database: database seed (en database server kan inneholde mange database instanser)

Dersom vi ikke ønsker å "hardkode" passord (skrive det i programmet i klar tekst eller i en fil), kan vi bruke modulen **getPass** for å taste inn passordet når programmet starter.


In [3]:
# importer funksjonen getpass fra modulen getpass.
# Vi bruker getpass for å lese passord fra bruker (slik at det ikke vises i klartekst)
from getpass import getpass

In [4]:
# Database connection object
global db_connection
db_connection = None

my_host = "mysql.stud.iie.ntnu.no"
my_user = "rouhani" # Skriv inn brukernavnet ditt her
my_password = getpass()
my_database = "rouhani"

def get_db_connection():
    if 'db_connection' in globals():
        db_connection = pymysql.connect(host = my_host, user = my_user, password = my_password, database = my_database)
    return db_connection

### Koble opp mot databasen og les fra en tabell

In [15]:


with get_db_connection() as my_db:
    my_cursor = my_db.cursor()

    my_sql = "SELECT * FROM objects"
    my_cursor.execute(my_sql)

    for row in my_cursor:
        print(row)



('TBM.0832', 'Stol', 1, '12.04.2023', 'Hanne', '12.04.2023', 'Kristin', 'Olsen Giver', 'PL.A1.B2', 'Tester')
('TBM.4892', 'Kommode', 1, '03.04.2011', 'Museumsbestyrer Olav N', '29.04.2012', 'Museumsbestyrer Olav Nilsen', 'Museumsbestyrer Olav Nilsen', 'M1.R2.P1.H1', '')


### Koble opp mot databasen og oppdater rader i en tabell


In [6]:
#Sjekk om regnr existerer i databasen
def reg_no_exist(reg_no,table_name):
    """
    Sjekk om regnnr existerer allerede i basen
    :param regnr: type string
    :return: boolean
    """
    with get_db_connection() as db:

        cursor = db.cursor()
        sql = "SELECT count(reg_no) as reg_no_count FROM {0} WHERE reg_no='{1}'".format(table_name,reg_no)
        cursor.execute(sql)

        reg_no_count = 0

        for row in cursor:
            reg_no_count=  row[0]

        if reg_no_count==0:
            return False
        else:
            return True

#Lagrer gjenstand i basen
def save_object_db(giver_val,
                      innlemmet_dato_val,
                      kategori_id,
                      kommentar_val,
                      mottatt_av_val,
                      navn_val,
                      plassering_val,
                      registrert_av_val,
                      registrerings_dato_val,
                      reg_no):
    """
    Lagre gjenstand info i databasen.
    Dersom den finnes allerede, blir gjenstand oppdatert
    Dersom den ikke finnes, blir den opprettet.
    :param giver_val: string
    :param innlemmet_dato_val: string
    :param kategori_id: int
    :param kommentar_val: string
    :param mottatt_av_val: string
    :param navn_val: string
    :param plassering_val: string
    :param registrert_av_val: string
    :param registrerings_dato_val: string
    :param regnr: string
    :return:
    """

    #****************************************
    # Oppgave 4.1
    # Fullfør innholdet i denne funksjonen!
    #****************************************

    with get_db_connection() as my_db:

        my_db.autocommit(True)
        cursor = my_db.cursor()

        if reg_no_exist(reg_no,'objects'):
            cursor.execute("UPDATE objects SET giver=%s,in_date=%s,category_id=%s,comment=%s,received_by=%s,name=%s,placement=%s,reg_by=%s,reg_date=%s WHERE reg_no=%s",
                        (giver_val,innlemmet_dato_val,kategori_id,kommentar_val,mottatt_av_val,navn_val,plassering_val,registrert_av_val,registrerings_dato_val,reg_no))
        else:
            cursor.execute("INSERT INTO objects (giver,in_date,category_id,comment,received_by,name,placement,reg_by,reg_date,reg_no) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",
                        (giver_val,innlemmet_dato_val,kategori_id,kommentar_val,mottatt_av_val,navn_val,plassering_val,registrert_av_val,registrerings_dato_val,reg_no))

#Oppdater eksisterende rad
save_object_db('Olsen Giver','12.04.2023', 1, 'Tester', 'Kristin', 'Stol', 'PL.A1.B2','Hanne','12.04.2023','TBM.0832')


#legg til ny rad
save_object_db('Olsen Giver','12.04.2023', 1, 'Tester', 'Kristin', 'Stol', 'PL.A1.B2','Hanne','12.04.2023','TBM.0832-N')


### Koble opp mot databasen og fjern rader fra en tabell

In [14]:
#Slett gjenstand
def delete_object(reg_no):
    """
    :param regnr: Registreriongsnr på gjenstand som skal slettes
    :return: ingen returverdi.

    Funksjonen skal finne gjenstanden og slette denne.
    1. Slett fra properties-tabellen for det gitte registreringsnummeret
    2. Slett fra provenances-tabellen for det gitte registreringsnummeret
    3. Slett fra objects-tabellen for det gitte registreringsnummeret

    Dersom sletting går bra, skriv en melding til skjermen.
    """

    #******************************************
    # Oppgave 4.3
    # Fullfør denne funksjonen
    #******************************************


    with get_db_connection() as db:
        db.autocommit(True)

        cursor = db.cursor()
        cursor.execute("DELETE from properties where reg_no='" + reg_no+"'")
        cursor.execute("DELETE from provenances where reg_no='" + reg_no+"'")
        cursor.execute("DELETE from objects where reg_no='" + reg_no+"'")


delete_object('TBM.0832-N')